# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room Part #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with [LangChain v0.2.0](https://python.langchain.com/v0.2/docs/versions/v0_2/)
  4. Synthetic Dataset Generation for Evaluation using the [Ragas](https://github.com/explodinggradients/ragas) framework.
  

- 🤝 Breakout Room Part #2:
  1. Evaluating our pipeline with Ragas
  3. Making Adjustments to our RAG Pipeline
  4. Evaluating our Adjusted pipeline against our baseline
  5. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

> NOTE: Using this notebook as presented will occur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step - if you want to reduce costs, please use the provided commented code to leverage `GPT-3.5-Turbo` as the `critic_llm`!

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room Part #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [1]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

We'll also get the "star of the show" today, which is Ragas!

In [2]:
!pip install -qU ragas

We'll be leveraging [QDrant](https://qdrant.tech/) again as our LangChain `VectorStore`.

We'll also install `pymupdf` and its dependencies which will allow us to load PDFs using the `PyMuPDFLoader` in the `langchain-community` package!

In [3]:
!pip install -qU qdrant-client pymupdf pandas

## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [4]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.1.0

Building on what we learned last week, we'll be leveraging LangChain v0.1.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.2.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [5]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

documents = loader.load()

In [6]:
documents[0].metadata

{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'page': 0,
 'total_pages': 195,
 'format': 'PDF 1.3',
 'title': 'The Pmarca Blog Archives',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': 'Mac OS X 10.10 Quartz PDFContext',
 'creationDate': "D:20150110020418Z00'00'",
 'modDate': "D:20150110020418Z00'00'",
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [8]:
len(documents)

1864

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [9]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a QDrant VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [10]:
from langchain_community.vectorstores import Qdrant

qdrant_vector_store = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="PMarca Blogs",
)

####❓ Question #1:

List out a few of the techniques that Qdrant uses that make it performant.

> NOTE: Check the [documentation](https://qdrant.tech/documentation/overview/) for more information about QDrant!

Qdrant's performance is driven by several key features, including ( but not limited to) the following:

-HNSW Algorithm: Uses the Hierarchical Navigable Small World (HNSW) algorithm for fast and scalable similarity searches. HNSW provides a balance between search accuracy and speed.
-Efficient Memory Use: Optimized for handling large datasets, ensuring quick responses.
-Parallel Processing: Leverages multi-core processors to manage multiple queries at once.

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [11]:
retriever = qdrant_vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [12]:
retrieved_documents = retriever.invoke("What is a rule of thumb for selecting an industry to invest in?")

In [13]:
for doc in retrieved_documents:
  print(doc)

page_content='the existing order — and make sure that those forces of change\nhave a reasonable chance at succeeding.\nSecond rule of thumb:\nOnce you have picked an industry, get right to the center of it' metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 125, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': 'fee58a1ec8ff41518821682dceedbd5e', '_collection_name': 'PMarca Blogs'}
page_content='Third rule:\nIn a rapidly changing Held like technology, the best place to\nget experience when you’re starting out is in younger, high-\ngrowth companies.' metadata={'source': 'https://d1lamhf6l6yk6d

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [14]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [15]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [16]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [17]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

####🏗️ Activity #1:

Describe the pipeline shown above in simple terms. You can include a diagram if desired.

The RAG chain:
(1) Takes the user question and retrieves relevant context, 
(2) Passes the context through unchanged, 
(3) Formats the prompt with context and question, then sends it to the LLM to generate a response, preserving the context.

Let's test it out!

In [20]:
question = "What is a rule of thumb for selecting an industry to invest in?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Get right to the center of the industry.


In [21]:
question = "What did Pink Floyd have to say about how to proceed when investing in a new industry?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

I don't know.
[Document(page_content='ask if you can call them again if things change.\nTrust me — they’d much rather be saying “yes” than “no” —\nthey need all the good investments they can get.\nSecond, consider the environment.', metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 15, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': '1a1233a735d347108a95976231fdb6aa', '_collection_name': 'PMarca Blogs'}), Document(page_content='watching carefully — if everyone agrees right up front that\nwhatever you are doing makes total sense, it probably isn’t a new\nand radical enough idea to justify a

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

## Task 4: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evaluating on every core metric today, but in order to do that - we'll need to create a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

In [22]:
loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

eval_documents = loader.load()

text_splitter_eval = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 50
)

eval_documents = text_splitter_eval.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?

So we don't mess up our test results. It helps ensure more diverse data variations and helps prevent overfitting.

In [23]:
len(eval_documents)

624


> NOTE: 🛑 Using this notebook as presented will occur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step - if you want to reduce costs, please use the provided commented code to leverage GPT-3.5-Turbo as the critic_llm. If you're attempting to create a lot of samples please be aware of cost, as well as rate limits. 🛑

In [24]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
# critic_llm = ChatOpenAI(model="gpt-3.5-turbo") <--- If you don't have GPT-4 access, or to reduce cost/rate limiting issues.
critic_llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

testset = generator.generate_with_langchain_docs(eval_documents, 20, distributions, is_async = False)
testset.to_pandas()

embedding nodes:   0%|          | 0/1248 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/20 [00:00<?, ?it/s]

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What are some signs that indicate the executiv...,[sureXre signal that the executive is not work...,nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,What is Los Angeles known for in terms of oppo...,[most interesting opportunity available — the ...,Los Angeles is known for opportunities and ind...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,How important is it for a startup to focus on ...,"[developing a large market, as opposed to Xght...",Focusing on developing a large market is impor...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,What are the potential risks and challenges fa...,[Here’s why you shouldn’t do that:\nWhat are t...,Not raising enough money risks the survival of...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,What are some techniques that can help entrepr...,[I want to tell you about my new startup” is a...,nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,What is the purpose of using temporary subfold...,[you can reply to a lot of messages with “I’m ...,The purpose of using temporary subfolders in e...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,How does the Liking/Loving Tendency influence ...,[One very practical consequence of Liking/Lovi...,The Liking/Loving Tendency influences entrepre...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,What is the writing style and genre of Peter H...,"[pandemics, nuclear terrorist attacks, governm...",The writing style of Peter Hamilton's work is ...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,How does the faster cycle time in startups mak...,[would have spent 10 or 20 or 30 years buildin...,The faster cycle time in startups makes it eas...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,How can funding to blog help an entrepreneur i...,[looking for funding to blog — about their sta...,Funding to blog can help an entrepreneur in th...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


####❓ Question #3:

`{simple: 0.5, reasoning: 0.25, multi_context: 0.25}`

What exactly does this mapping refer to?

> NOTE: Check out the Ragas documentation on this generation process [here](https://docs.ragas.io/en/stable/concepts/testset_generation.html).

It shows what share of questions are rewritten in the following ways vs. kept as-is (and not evolved): 

"Reasoning: Rewrite the question in a way that enhances the need for reasoning to answer it effectively.

Multi-Context: Rephrase the question in a manner that necessitates information from multiple related sections or chunks to formulate an answer."

Let's look at the output and see what we can learn about it!

In [25]:
testset.test_data[0]

DataRow(question='What are some signs that indicate the executive is not working out and may need to be fired?', contexts=['sureXre signal that the executive is not working out and proba-\nbly needs to be Xred. More on that below.\nThird, ruthlessly violate the chain of command in order to gather data.\nI don’t mean going around telling people under an executive\nwhat to do without her knowing about it. I mean, ask questions,\ncontinually, at all levels of the organization. How are things\ngoing? What do you think of the new hires? How oaen are you\nmeeting with your manager? And so on.\nYou never want the bulk of your information about a function\ncoming from the executive running that function. That’s the'], ground_truth='nan', evolution_type='simple', metadata=[{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 65, 'total_pages': 19

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [26]:
test_df = testset.to_pandas()

In [27]:
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What are some signs that indicate the executiv...,[sureXre signal that the executive is not work...,nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,What is Los Angeles known for in terms of oppo...,[most interesting opportunity available — the ...,Los Angeles is known for opportunities and ind...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,How important is it for a startup to focus on ...,"[developing a large market, as opposed to Xght...",Focusing on developing a large market is impor...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,What are the potential risks and challenges fa...,[Here’s why you shouldn’t do that:\nWhat are t...,Not raising enough money risks the survival of...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,What are some techniques that can help entrepr...,[I want to tell you about my new startup” is a...,nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,What is the purpose of using temporary subfold...,[you can reply to a lot of messages with “I’m ...,The purpose of using temporary subfolders in e...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,How does the Liking/Loving Tendency influence ...,[One very practical consequence of Liking/Lovi...,The Liking/Loving Tendency influences entrepre...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,What is the writing style and genre of Peter H...,"[pandemics, nuclear terrorist attacks, governm...",The writing style of Peter Hamilton's work is ...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,How does the faster cycle time in startups mak...,[would have spent 10 or 20 or 30 years buildin...,The faster cycle time in startups makes it eas...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,How can funding to blog help an entrepreneur i...,[looking for funding to blog — about their sta...,Funding to blog can help an entrepreneur in th...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


In [28]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [29]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [30]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [31]:
response_dataset[0]

{'question': 'What are some signs that indicate the executive is not working out and may need to be fired?',
 'answer': 'Some signs that indicate the executive is not working out and may need to be fired include ruthlessly violating the chain of command in order to gather data, having areas where they are deficient in judgment, and the potential to permanently cripple the function of the company.',
 'contexts': ['sureXre signal that the executive is not working out and proba-\nbly needs to be Xred. More on that below.\nThird, ruthlessly violate the chain of command in order to gather data.',
  'him. An executive doesn’t work out, it can — worst case — per-\nmanently cripple her function and sometimes the entire com-\npany. Therefore, it is far more important to Fre a bad executive as fast',
  'manage executives. It turns out that just about every executive\nin the world has a few things that are seriously wrong with\nthem. They have areas where they are truly deXcient in judg-',
  'dev

# 🤝 Breakout Room Part #2

## Task 1: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [32]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [33]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

In [34]:
results

{'faithfulness': 0.7275, 'answer_relevancy': 0.9079, 'context_recall': 0.5858, 'context_precision': 0.6681, 'answer_correctness': 0.4880}

In [35]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What are some signs that indicate the executiv...,Some signs that indicate the executive is not ...,[sureXre signal that the executive is not work...,nan,0.800000,1.000000,0.000000,0.000000,0.181586
1,What is Los Angeles known for in terms of oppo...,Los Angeles is known for opportunities and ind...,[the city where all the action is happening.\n...,Los Angeles is known for opportunities and ind...,0.500000,1.000000,1.000000,0.000000,1.000000
2,How important is it for a startup to focus on ...,It is important for a startup to focus on deve...,"[competitor, be sure to take a step back and s...",Focusing on developing a large market is impor...,1.000000,0.963745,1.000000,1.000000,0.747455
3,What are the potential risks and challenges fa...,Not raising enough money risks the survival of...,[Here’s why you shouldn’t do that:\nWhat are t...,Not raising enough money risks the survival of...,1.000000,0.921043,0.333333,0.805556,0.354155
4,What are some techniques that can help entrepr...,Some techniques that can help entrepreneurs en...,[If\nyou\nengage\nin\na\nset\nof\nthese\ntechn...,nan,1.000000,1.000000,0.000000,0.000000,0.180523
5,What is the purpose of using temporary subfold...,To organize projects or pressing issues that n...,"[matter, right now.\nThose subfolders then get...",The purpose of using temporary subfolders in e...,0.000000,0.887947,1.000000,0.500000,0.702416
6,How does the Liking/Loving Tendency influence ...,The Liking/Loving Tendency influences entrepre...,[other facts to facilitate love.\nThe applicat...,The Liking/Loving Tendency influences entrepre...,0.666667,0.969881,1.000000,0.583333,0.458324
7,What is the writing style and genre of Peter H...,The writing style of Peter Hamilton's work is ...,[ders and executions around the clock. Haven’t...,The writing style of Peter Hamilton's work is ...,0.666667,0.964951,1.000000,1.000000,0.540322
8,How does the faster cycle time in startups mak...,The faster cycle time in startups makes it eas...,[product families or grow market share. This o...,The faster cycle time in startups makes it eas...,0.666667,0.971224,0.000000,0.805556,0.491839
9,How can funding to blog help an entrepreneur i...,Funding to blog can help an entrepreneur in th...,"[esting things going on, about their point of ...",Funding to blog can help an entrepreneur in th...,0.750000,0.997717,0.500000,1.000000,0.412411


## Task 2: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

> NOTE: MultiQueryRetriever is expanded on [here](https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever) but for now, the implementation is not important to our lesson!

In [36]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [37]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [38]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [39]:
response = retrieval_chain.invoke({"input": "Who is Taylor Swift fueding with?"})

In [40]:
print(response["answer"])

I'm sorry, but based on the context provided, I do not have information about any feud involving Taylor Swift.


In [41]:
response = retrieval_chain.invoke({"input": "Why are they fueding?"})

In [42]:
print(response["answer"])

The text does not provide any information about a feud or conflict between individuals or groups.


Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [43]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [44]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

In [46]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

In [47]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What are some signs that indicate the executiv...,Some signs that indicate the executive is not ...,[sureXre signal that the executive is not work...,nan,1.000000,0.000000,0.000000,0.000000,0.182243
1,What is Los Angeles known for in terms of oppo...,Los Angeles is known for opportunities and ind...,[the city where all the action is happening.\n...,Los Angeles is known for opportunities and ind...,1.000000,1.000000,1.000000,1.000000,0.669579
2,How important is it for a startup to focus on ...,It is very important for a startup to focus on...,"[answer, in part because in the beginning of a...",Focusing on developing a large market is impor...,0.600000,0.980525,1.000000,1.000000,0.604529
3,What are the potential risks and challenges fa...,Not raising enough money risks the survival of...,[Here’s why you shouldn’t do that:\nWhat are t...,Not raising enough money risks the survival of...,0.666667,0.921043,1.000000,0.805556,0.514422
4,What are some techniques that can help entrepr...,Some techniques that can help entrepreneurs en...,[priate for your particular startup.\nWith a w...,nan,0.750000,1.000000,0.400000,0.325000,0.178016
5,What is the purpose of using temporary subfold...,The purpose of using temporary subfolders in e...,"[matter, right now.\nThose subfolders then get...",The purpose of using temporary subfolders in e...,0.666667,1.000000,1.000000,0.416667,0.454245
6,How does the Liking/Loving Tendency influence ...,The Liking/Loving Tendency influences entrepre...,[other facts to facilitate love.\nThe applicat...,The Liking/Loving Tendency influences entrepre...,0.500000,1.000000,1.000000,0.250000,0.394937
7,What is the writing style and genre of Peter H...,Peter Hamilton's work is described as large-sc...,[ders and executions around the clock. Haven’t...,The writing style of Peter Hamilton's work is ...,1.000000,0.922061,1.000000,0.700000,0.484209
8,How does the faster cycle time in startups mak...,The faster cycle time in startups makes it eas...,[product families or grow market share. This o...,The faster cycle time in startups makes it eas...,0.166667,0.946895,0.000000,0.826020,0.407366
9,How can funding to blog help an entrepreneur i...,Funding a blog can help an entrepreneur in the...,"[esting things going on, about their point of ...",Funding to blog can help an entrepreneur in th...,0.750000,0.997438,1.000000,1.000000,0.567418


## Task 3: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [48]:
results

{'faithfulness': 0.7275, 'answer_relevancy': 0.9079, 'context_recall': 0.5858, 'context_precision': 0.6681, 'answer_correctness': 0.4880}

And see how our advanced retrieval modified our chain!

In [49]:
advanced_retrieval_results

{'faithfulness': 0.7379, 'answer_relevancy': 0.9194, 'context_recall': 0.7896, 'context_precision': 0.6802, 'answer_correctness': 0.5338}

In [50]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,0.727500,0.737912,0.010412
1,answer_relevancy,0.907909,0.919358,0.011449
2,context_recall,0.585833,0.789583,0.203750
3,context_precision,0.668056,0.680171,0.012116
4,answer_correctness,0.487997,0.533765,0.045768


## Task 4: Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #2:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

In [51]:
# Create embeddings with openai text-embedding-3-small model
new_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [52]:
# Create an in-memory Qdrant vector store with the new embeddings
vector_store = Qdrant.from_documents(
    documents,
    new_embeddings,
    location=":memory:",
    collection_name="PMarca Blogs - TE3 - MQR",
)

In [53]:
# Create a new retriever from the vector store
new_retriever = vector_store.as_retriever()

In [54]:
# Create a new advanced retriever with the new retriever and the primary_qa_llm
new_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

In [55]:
# Create a new retrieval chain with the new advanced retriever and the document_chain
new_retrieval_chain = create_retrieval_chain(new_advanced_retriever, document_chain)

In [56]:
# Create a new response dataset with the new retrieval chain
answers = []
contexts = []

# Loop through the test questions and get the answers and contexts
for question in test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [57]:
# Create a new response dataset with the new answers and contexts

new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [58]:
# Evaluate the new response dataset with the metrics

new_advanced_retrieval_results = evaluate(new_response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

In [59]:
# Convert the new advanced retrieval results to a pandas dataframe 
new_advanced_retrieval_results

{'faithfulness': 0.7453, 'answer_relevancy': 0.9570, 'context_recall': 0.6533, 'context_precision': 0.6635, 'answer_correctness': 0.5328}

In [61]:
# Compare the original results with the new advanced retrieval results

# Convert the results to a pandas dataframe
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'ADA + Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA + MQR'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'TE3 + MQR'])

# Merge the dataframes
df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

# Calculate the deltas
df_merged['ADA + MQR -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + MQR']
df_merged['Baseline -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + Baseline']

# Display the results
df_merged

,Metric,ADA + Baseline,ADA + MQR,TE3 + MQR,ADA + MQR -> TE3 + MQR,Baseline -> TE3 + MQR
0,faithfulness,0.727500,0.737912,0.745278,0.007366,0.017778
1,answer_relevancy,0.907909,0.919358,0.957024,0.037666,0.049115
2,context_recall,0.585833,0.789583,0.653333,-0.136250,0.067500
3,context_precision,0.668056,0.680171,0.663456,-0.016715,-0.004599
4,answer_correctness,0.487997,0.533765,0.532839,-0.000927,0.044842


####❓ Question #4:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

I'd want to test this a few more times but initial indications are that there are mixed improvements but test-embedding-3-small is not categorically better across the board.

Based on this eval, it looks like text-embedding-3-small outperforms ada on faithfulness and answer relevancy but is worse on context recall and context precision. Answer correctness seems roughly the same for both. 

## BONUS ACTIVITY: Using a Better Generator

Now that we've seen how much more effective a better Retrieval pipeline is, let's look at what impact a better(?) Generator is!

Adapt the above `TE3 + MQR` pipeline to use `GPT-4o` and compare the results below!